## About
This notebook is a work in progress to fix https://github.com/singularity-energy/open-grid-emissions/issues/195 and https://github.com/singularity-energy/open-grid-emissions/issues/163

In [ ]:
# import packages
import pandas as pd
import os

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930

year = 2020
path_prefix = f"{year}/"

## The plan
1. Create primary fuel table
2. Assign subplant id to primary fuel table
3. Calculate fuel weighted emission factor for each subplant_id
4. fill missing using fuel weight


We still need to assign a fuel code to each observation in CEMS
1. Calculate a subplant primary fuel as well (based on allocated EIA-923)
2. Then use PSDC
3. Then fill based on single fuel plants

Assign 

In [ ]:
# 3. Clean EIA-923 Generation and Fuel Data at the Monthly Level
####################################################################################
print("3. Cleaning EIA-923 data")
eia923_allocated, primary_fuel_table = data_cleaning.clean_eia923(year, False)

In [ ]:
subplant_crosswalk = pd.read_csv(
            outputs_folder(f"{year}/subplant_crosswalk.csv"),
            dtype=get_dtypes(),
        )

subplant_crosswalk

In [ ]:
# calculate subplant weighted emission factor
weighted_ef = eia923_allocated.groupby(["plant_id_eia","subplant_id","report_date",], dropna=False)[["fuel_consumed_mmbtu","co2_mass_lb","ch4_mass_lb","n2o_mass_lb","nox_mass_lb","so2_mass_lb"]].sum().reset_index()

# drop any observations with zero fuel consumption
weighted_ef = weighted_ef[weighted_ef["fuel_consumed_mmbtu"] > 0]


for pollutant in ["co2","ch4","n2o","nox","so2"]:
    weighted_ef[f"{pollutant}_ef_lb_per_mmbtu"] = weighted_ef[f"{pollutant}_mass_lb"] / weighted_ef["fuel_consumed_mmbtu"]
weighted_ef


In [ ]:
weighted_ef[weighted_ef["plant_id_eia"] == 3]